<a href="https://colab.research.google.com/github/tanupriya9102/metro_cc/blob/main/BertChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from google.colab import files
upload=files.upload()

Saving newchatbot.xlsx to newchatbot.xlsx


In [ ]:
# Preprocess the data
questions = df["Question"].values
answers = df["Answer"].values

In [ ]:
# Set up BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from sklearn.metrics import accuracy_score

# Load the data from Excel file
df = pd.read_excel("newchatbot.xlsx")  # Adjust the file path accordingly

# Preprocess the data
questions = df["Question"].values
answers = df["Answer"].values

# Set up BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the input data
input_ids = []
attention_masks = []
start_positions = []
end_positions = []

for question, answer in zip(questions, answers):
    encoded_dict = tokenizer.encode_plus(
        question,
        answer,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

    # Retrieve the start position
    start_position = encoded_dict['input_ids'].tolist().index(102) if 102 in encoded_dict['input_ids'].tolist() else -1
    start_positions.append(torch.tensor(start_position))

    # Retrieve the end position
    end_position = encoded_dict['input_ids'].tolist().index(103) if 103 in encoded_dict['input_ids'].tolist() else -1
    end_positions.append(torch.tensor(end_position))

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
start_positions = torch.stack(start_positions, dim=0)  # Convert the list of tensors to a tensor
end_positions = torch.stack(end_positions, dim=0)  # Convert the list of tensors to a tensor

# Create a custom dataset
class ChatbotDataset(Dataset):
    def __init__(self, input_ids, attention_masks, start_positions, end_positions):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.start_positions = start_positions
        self.end_positions = end_positions

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'start_positions': self.start_positions[idx],
            'end_positions': self.end_positions[idx]
        }

dataset = ChatbotDataset(input_ids, attention_masks, start_positions, end_positions)

# Model Configuration
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training Loop
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

optimizer = AdamW(model.parameters(), lr=1e-5)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Convert logits to predictions
        start_logits = outputs.start_logits.detach().cpu().numpy()
        end_logits = outputs.end_logits.detach().cpu().numpy()
        preds = [tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(ids)) for ids in zip(*[start_logits.argmax(axis=1), end_logits.argmax(axis=1)])]
        all_preds.extend(preds)
        all_labels.extend([tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(ids)) for ids in zip(*[start_positions.cpu().numpy(), end_positions.cpu().numpy()])])

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")

# Evaluation
# ...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 1 - Average Loss: 2.5983 - Accuracy: 0.0000
Epoch 2 - Average Loss: 0.3081 - Accuracy: 0.0000
Epoch 3 - Average Loss: 0.0416 - Accuracy: 0.0000
